# Repeat Experiment 
## import packages

In [ ]:
import warnings
import torch

warnings.filterwarnings("ignore")

In [ ]:
from tcnn.utils.experiment.repeat import repeat_experiment

In [ ]:
from os.path import join
import torch

dataset_name = "urban8k"
config = {
    "order": "01",
    "task": "multiclass",
    "data": {
        "batch_size": 32,
        "size": 32000,
    },
    "network": {"input_channels": 1, "linear_size": 127680, "num_classes": 10},
    "train": {
        "criterion": torch.nn.CrossEntropyLoss(),
        "checkpoint_save_dir": "checkpoints",
        "epochs": 200,
        "device": "cuda" if torch.cuda.is_available() else "cpu",
    },
    "repeat": {"num_experiments": 5, "epochs_per_experiemnt": 200, "log_save_dir": "logs"},
}

In [ ]:
ORDER = config["order"]
SIZE = config["data"]["size"]
TASK = config["task"]
NUM_CLASSES = config["network"]["num_classes"]
EPOCHS = config["train"]["epochs"]
BATCH_SIZE = config["data"]["batch_size"]
INPUT_CHANNEL = config["network"]["input_channels"]
LINEAR_SIZE = config["network"]["linear_size"]
CRITERION = config["train"]["criterion"]
DEVICE = config["train"]["device"]
NUM_EXPERIMENTS = config["repeat"]["num_experiments"]
EPOCHS_PER_EXPERIMENT = config["repeat"]["epochs_per_experiemnt"]
experiment_name = (
    f"repeat-{ORDER}-{dataset_name}-{SIZE}-{NUM_EXPERIMENTS}-{EPOCHS_PER_EXPERIMENT}"
)

In [ ]:
checkpoint_save_dir = join(config["train"]["checkpoint_save_dir"], experiment_name)
log_save_dir = join(config["repeat"]["log_save_dir"])

## get dataset

In [ ]:
from tcnn.utils.simulation.dataset.urban_sound_8k import UrbanSound8K
from sklearn.model_selection import train_test_split
import numpy as np

urban_sound_dataset = UrbanSound8K()
x, y = urban_sound_dataset.get_all_data()

x = np.transpose(x, (0, 2, 1))

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=2024
)


print("Loading UrbanSound8K dataset:")
print(f"Train set size: {x_train.shape[0]} Training set shape: { x_train.shape}")
print(f"Test set size: {x_test.shape[0]} Test set shape: { x_test.shape}")
print("done!")
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)

x_test = torch.tensor(x_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

train_dataset = torch.utils.data.TensorDataset(x_train, y_train)
test_dataset = torch.utils.data.TensorDataset(x_test, y_test)

## reset dataloader

In [ ]:
def reset_dataloader(dataset, batch_size, shuffle=True):
    return torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
    )

## import models

In [ ]:
import lenet_models

model_dict = lenet_models.get_model_dict(INPUT_CHANNEL, NUM_CLASSES, LINEAR_SIZE)

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau  # type: ignore


def def_optimizer(model):
    return torch.optim.Adam(model.parameters())


def def_scheduler(optimizer):
    return ReduceLROnPlateau(
        optimizer, mode="max", factor=0.5, patience=10, min_lr=0.0001, verbose=True
    )

## define result dict

In [ ]:
import torch

results = dict()

## repeat experiment

In [ ]:
for model_name, model in model_dict.items():
    print("Traing model: ", model_name)
    results[model_name] = repeat_experiment(
        model,
        [train_dataset, test_dataset],
        reset_dataloader,
        BATCH_SIZE,
        CRITERION,
        def_optimizer,
        def_scheduler,
        NUM_EXPERIMENTS,
        EPOCHS_PER_EXPERIMENT,
        experiment_name + "-" + model_name,
        checkpoint_save=True,
        checkpoint_save_dir=checkpoint_save_dir,
        scheduler_sign="val_acc",
    
    )
    with torch.no_grad():
        torch.cuda.empty_cache()
    print("***" * 10 + f"{model_name} done" + "***" * 10)

In [ ]:
from tcnn.utils.experiment.log import show_repeat_result, save_result

show_repeat_result(results)
save_result(results, save_name=experiment_name)

In [ ]:
# from tcnn.utils.experiment.plot import plot_experiment_errorbar

# plot_experiment_errorbar(
#     results, metric_key="accuracy", baseline_key="lenet", ylabel="Accuracy"
# )

In [ ]:
# plot_experiment_errorbar(
#     results, metric_key="accuracy", baseline_key="lenet_relu", ylabel="Accuracy"
# )

In [ ]:
# plot_experiment_errorbar(
#     results, metric_key="auc_score", baseline_key="lenet", ylabel="AUC Score"
# )

In [ ]:
# plot_experiment_errorbar(
#     results, metric_key="auc_score", baseline_key="lenet_relu", ylabel="AUC Score"
# )

In [ ]:
# plot_experiment_errorbar(
#     results, metric_key="f1", baseline_key="lenet", ylabel="F1 Score"
# )

In [ ]:
# plot_experiment_errorbar(
#     results, metric_key="f1", baseline_key="lenet_relu", ylabel="F1 Score"
# )

In [ ]:
import torch

with torch.no_grad():
    torch.cuda.empty_cache()